<a href="https://colab.research.google.com/github/Lakshmiaishwarya01/Sproutify/blob/main/Sproutify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin pandas numpy scikit-learn

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import time

def initialize_firebase():
    """Initialize Firebase Admin SDK"""
    cred = credentials.Certificate(
        "/content/firebase-sdk.json"
    )
    firebase_admin.initialize_app(cred, {
        'databaseURL': "https://sproutify-3a23c-default-rtdb.asia-southeast1.firebasedatabase.app
    })

def load_historical_data():
    """Load and preprocess historical training data"""
    # Example historical data structure (replace with your actual dataset)
    money_plant_data = {
        'moisture': [45, 30, 25, 50, 35, 40, 28, 55, 42, 38],
        'temperature': [25, 28, 27, 24, 26, 29, 25, 23, 26, 27],
        'humidity': [65, 60, 55, 70, 62, 58, 63, 68, 64, 61],
        'pump_status': [1, 1, 1, 0, 1, 1, 1, 0, 0, 1]  # Historical decisions
    }

    water_lily_data = {
        'moisture': [60, 55, 50, 65, 58, 52, 48, 62, 57, 53],
        'temperature': [24, 26, 25, 23, 25, 27, 24, 22, 25, 26],
        'humidity': [70, 68, 65, 75, 69, 67, 66, 72, 68, 67],
        'pump_status': [1, 0, 1, 0, 1, 1, 1, 0, 1, 1]  # Historical decisions
    }

    return pd.DataFrame(money_plant_data), pd.DataFrame(water_lily_data)

def train_models(money_plant_df, water_lily_df):
    """Train separate models for each plant"""
    models = {}

    for plant_name, df in [("money_plant", money_plant_df), ("water_lily", water_lily_df)]:
        X = df[['moisture', 'temperature', 'humidity']]
        y = df['pump_status']

        model = LinearRegression()
        model.fit(X, y)
        models[plant_name] = model

    return models

def get_current_sensor_data():
    """Fetch current sensor readings from Firebase"""
    try:
        sensor_ref = db.reference('sensors')
        sensor_data = sensor_ref.get()

        if not sensor_data:
            raise ValueError("No sensor data available")

        return {
            'money_plant': {
                'moisture': sensor_data.get('moneyPlant', {}).get('moisture', 0),
                'temperature': sensor_data.get('moneyPlant', {}).get('temperature', 0),
                'humidity': sensor_data.get('moneyPlant', {}).get('humidity', 0)
            },
            'water_lily': {
                'moisture': sensor_data.get('waterLily', {}).get('moisture', 0),
                'temperature': sensor_data.get('waterLily', {}).get('temperature', 0),
                'humidity': sensor_data.get('waterLily', {}).get('humidity', 0)
            }
        }
    except Exception as e:
        print(f"Error reading sensor data: {e}")
        return None

def predict_pump_status(models, sensor_data):
    """Make predictions for both plants"""
    predictions = {}

    for plant_name in ['money_plant', 'water_lily']:
        plant_data = sensor_data[plant_name]
        features = np.array([[
            plant_data['moisture'],
            plant_data['temperature'],
            plant_data['humidity']
        ]])

        # Get prediction and convert to binary
        pred = models[plant_name].predict(features)[0]
        predictions[plant_name] = 1 if pred >= 0.5 else 0

    return predictions

def update_pump_controls(predictions):
    """Update Firebase pump controls based on predictions"""
    try:
        pump_ref = db.reference('PumpControl')

        # Update pump status for both plants
        pump_ref.update({
            'moneyPlant': {
                'Pump2': predictions['money_plant'],  # Water pump
                'Pump3': predictions['money_plant']   # Nutrient pump
            },
            'waterLily': {
                'Pump2': predictions['water_lily'],   # Water pump
                'Pump3': predictions['water_lily']    # Nutrient pump
            }
        })

        print("Pump controls updated successfully")
        print(f"Money Plant Pumps: {'ON' if predictions['money_plant'] else 'OFF'}")
        print(f"Water Lily Pumps: {'ON' if predictions['water_lily'] else 'OFF'}")

    except Exception as e:
        print(f"Error updating pump controls: {e}")

def main():
    # Initialize Firebase
    initialize_firebase()

    # Load and prepare historical data
    money_plant_df, water_lily_df = load_historical_data()

    # Train models
    models = train_models(money_plant_df, water_lily_df)

    # Main control loop
    while True:
        try:
            # Get current sensor readings
            sensor_data = get_current_sensor_data()

            if sensor_data:
                # Make predictions
                predictions = predict_pump_status(models, sensor_data)

                # Update pump controls
                update_pump_controls(predictions)

            # Wait before next iteration (e.g., every 5 minutes)
            time.sleep(300)

        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(60)  # Wait a minute before retrying

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Pump controls updated successfully
Money Plant Pumps: ON
Water Lily Pumps: ON


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Pump controls updated successfully
Money Plant Pumps: ON
Water Lily Pumps: ON


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


...

Pump controls updated successfully
Money Plant Pumps: ON
Water Lily Pumps: ON


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Pump controls updated successfully
Money Plant Pumps: ON
Water Lily Pumps: ON


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Pump controls updated successfully
Money Plant Pumps: ON
Water Lily Pumps: ON
